## Pre-requisite: Library requirements

In [55]:
# # !nvidia-smi
%pip install numpy
%pip install matplotlib
%pip install glob
%pip install os 
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
#!nvidia-smi
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import glob
import os

In [57]:
%ls
%ls ../data
%ls ../data/frac_map | head -10

0_Cluster_Rivers.jpeg/                   8523-Ying.ipynb*
8523_Common.ipynb*                       8523_Ying_UCL_Experiments.ipynb
8523_data_processing_70.ipynb            8523_Ying_UCL_Experiments-Task1.ipynb
8523_data_processing_80.ipynb            core.3755254*
8523_data_processing_90.ipynb            core.3779393*
8523_data_processing_kmeans_taks1.ipynb  core.3780321*
8523_Shonal2.ipynb*                      explanation.jpg*
8523-Shonal.ipynb*                       model_training_history.png
8523-Tanisha.ipynb*                      README.md*
8523_Task2_RTL_Exp_100_317.ipynb*        Shonal3.ipynb*
8523_Task2_RTL_Exp_70_2726.ipynb         Shonal4.ipynb*
8523_Task2_RTL_Exp_90_1940.ipynb         Shonal5.ipynb*
8523_Task2_UCL_Exp_100_317.ipynb         Shonal6_visualization.ipynb*
8523_Task2_UCL_Exp_70_2726.ipynb         Shonal7_fixing80.ipynb
8523_Task2_UCL_Exp_90_1940.ipynb         split_dataset.ipynb*
8523_Task2_UCL_Pipeline_Testing.ipynb*   TODO For Task 2
8523_Task2_Visualization.i

In [58]:
fracmaps_dir = '../data/frac_map'
frac_maps = os.listdir(fracmaps_dir)
timeseries_dir = '../data/time_series'
time_series = os.listdir(timeseries_dir)

total_maps = len(frac_maps) # 26754
total_ts = len(time_series) # 26754
print("total_maps len is ", total_maps, "total_ts len is ", total_ts)

## save fraction map and time series into img arrays
for i in range(0, total_maps):
    fm = frac_maps[i]
    frac_maps[i] = np.load(os.path.join(fracmaps_dir, fm), allow_pickle=True)
    ts = time_series[i]
    time_series[i] = np.load(os.path.join(timeseries_dir, ts), allow_pickle=True)
    

print(frac_maps[0].shape) ## (1,64,64)
print(time_series[0].shape) ## (442, 1)

total_maps len is  26754 total_ts len is  26754
(1, 64, 64)
(442, 1)


In [59]:
t = os.listdir('../data/frac_map')
t[0]

'ID_742140_frac_map.npy'

In [60]:
ID_labels = '../label_info/all_IDs_labels.npy'
ID_labels = np.load(ID_labels, allow_pickle=True)

# size of all labels
print(ID_labels.shape) ## (1000000,)

# check size of each label
# print(np.where(ID_labels==0)[0].shape) # (979799,) - unknown
# print(np.where(ID_labels==1)[0].shape) # (427,) - farm
# print(np.where(ID_labels==2)[0].shape) # (18707,) - reservoir
# print(np.where(ID_labels==3)[0].shape) # (317,) - river
# print(np.where(ID_labels==4)[0].shape) # (143,) - stable lakes
# print(np.where(ID_labels==5)[0].shape) # (288,) - seasonal lakes
# print(np.where(ID_labels==6)[0].shape) # (52,) - highly seasonal lakes
# print(np.where(ID_labels==7)[0].shape) # (255) - ephemeral lakes
# print(np.where(ID_labels==8)[0].shape) # (12,) - river runoff/oxbow

# fm_river_labels = np.where(ID_labels==3)[0]
# fm_river_labels
# print(fm_river_labels)
# fname = '../data/frac_map/ID_409869_frac_map.npy'
# s = np.load(fname, allow_pickle=True)
# river_0 = np.squeeze(s, axis = 0)
# plt.imshow(river_0)

(1000000,)


#### - Save all river fraction maps in the fm_rivers and river time series in the ts_rivers array

In [61]:
# extract river from fraction maps and time series maps
river_ids = np.where(ID_labels==3)[0]
print(river_ids.shape) # 317
#fracmaps_dir = '../data/frac_map'
#timeseries_dir = '../data/time_series'
fm_rivers = []
ts_rivers = []
pre = 'ID_'
fm_post = '_frac_map.npy'
ts_post = '_time_series.npy'

for sid in river_ids:
    fm_name = pre + str(sid) + fm_post
    ts_name = pre + str(sid) + ts_post
    fm_path = os.path.join(fracmaps_dir, fm_name)
    ts_path = os.path.join(timeseries_dir, ts_name)
    fm_rivers.append(np.load(fm_path, allow_pickle=True))
    ts_rivers.append(np.load(ts_path, allow_pickle=True))
#     print(np.load(ts_path, allow_pickle=True).shape) # (442,1)

fm_rivers = np.array(fm_rivers)
ts_rivers = np.array(ts_rivers)
print(fm_rivers.shape) # should be 317
print(ts_rivers.shape) # should be 317

# visualize the first river fraction map
# river_0 = np.squeeze(fm_rivers[0], axis = 0)
# plt.imshow(river_0)

(317,)
(317, 1, 64, 64)
(317, 442, 1)


# Preprocessing frac_maps

Currently, we have 26754 fraction maps. Each of these is of shape 1x64x64. Let's change that to 64x64 and aggregate them all into one array where each value is tuple in the form (name of file, 64x64 array)

In [62]:
fms = [] # fraction map: contains each 26754 64*64 array

for i in range(0, len(frac_maps)):
    fms.append(frac_maps[i][0])
    
fms = np.array(fms)
# print(fms[0])
# plt.imshow(fms[0])

In [63]:
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.utils import shuffle
import re
import cv2
from kmeanstf import KMeansTF

In [64]:
fm_filename = os.listdir('../data/frac_map') # has a 1-1 mapping with fms => fm_filename[i] is the filename of fms[i]
print(fm_filename[0])
ID_labels[0] # ID_labels is the ground truth array, len = 100,0000, ID_labels[ID] = class_no, ID is from filename

ID_742140_frac_map.npy


0

#### x_train is fms[i] which is a 64*64 2d array, y_train is ID_labels[getID(fm_filename, i)] which is a number

In [65]:
# a helper function which gets the ID of a fraction map array given the idx

def getID(fm_fn, idx):
    fn = fm_fn[idx] # get the file name
    fm_id = int(re.findall(r'\d+', fn)[0])
    return fm_id

getID(fm_filename, 1)

741138

In [66]:
y_train = []
for i in range(0, len(fms)):
    y_train.append(ID_labels[getID(fm_filename, i)])

y_train = np.array(y_train)
print(len(y_train))
print(np.unique(y_train))

26754
[0 1 2 3 4 5 6 7 8]


In [67]:
## check y_train label distribution 
print(np.where(y_train==0)[0].shape)
print(np.where(y_train==1)[0].shape)
print(np.where(y_train==2)[0].shape)
print(np.where(y_train==3)[0].shape)
print(np.where(y_train==4)[0].shape)
print(np.where(y_train==5)[0].shape)
print(np.where(y_train==6)[0].shape)
print(np.where(y_train==7)[0].shape)
print(np.where(y_train==8)[0].shape)

(24682,)
(427,)
(578,)
(317,)
(143,)
(288,)
(52,)
(255,)
(12,)


Since data is already between 0 to 1 we do not require normalization.

In [68]:
print(fms.shape)
print(y_train.shape)
print(fms[0].shape)
print(y_train[1])

(26754, 64, 64)
(26754,)
(64, 64)
7


In [69]:
# Reshaping input data
X_train = fms.reshape(len(fms),-1)
#X_test = x_test.reshape(len(x_test),-1)
print(X_train.shape)
# training, 

(26754, 4096)


The output is (26754, 4096). (64 x 64 = 4096)

## Setting the data

In [70]:
# numpy.random.shuffle(fms)
# training, test = fms[:80,:], dictionary[80:,:]
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(fms, y_train, test_size=0.2, random_state=42)
# print(X_train[0].shape) # 64*64
# print(X_train.shape) # 64*64

In [71]:
labeled = [] # has labels 1,2,3,4,5,6,7,8
y_labeled = [] # corresponding label
unlabeled = [] # anything else (24682, )
x_river = [] # holding rivers 317

print(len(fms)) # 26754 , 0 - 8

for i in range(0, len(fms)):

    label = ID_labels[getID(fm_filename, i)]
    
    if (label == 3):
        x_river.append(fms[i])

    if (label == 0): # unknown 
        unlabeled.append(fms[i])
    else:
        labeled.append(fms[i])
        y_labeled.append(label)
      
    
labeled = np.array(labeled)
unlabeled = np.array(unlabeled)
x_river = np.array(x_river)
print(labeled.shape)
print(len(y_labeled))
print(unlabeled.shape)
print(x_river.shape)


# # save into data/ying_unlabeled_train folder
# for i in range(0, len(unlabeled)):
#     img_name = "../data/ying_unlabeled_train/" + str(i) + ".jpeg"
#     matplotlib.image.imsave(img_name, unlabeled[i])

    
# # save into data/ying_labeled_test folder
# for i in range(0, len(labeled)):
#     img_name = "../data/ying_labeled_test/" + str(i) + ".jpeg"
#     matplotlib.image.imsave(img_name, labeled[i])

26754
(2072, 64, 64)
2072
(24682, 64, 64)
(317, 64, 64)


In [72]:
### training dataset: unlabeled
### river test dataset: x_river

# Reshape input dataset as required by resnet50v2
from skimage.transform import resize
im = np.reshape(unlabeled,(24682,64,64,1))
im = resize(im, (24682,64,64,3))
im.shape
X_train = im


# Reshape input dataset as required by resnet50v2
im = np.reshape(x_river,(317,64,64,1))
im = resize(im, (317,64,64,3))
im.shape
X_test = im


## now we use unlabeled to fit into the ucl model and train the mode, then fit into the x_river to calculate the accuracy

In [73]:
## some helper function 
def visualize(data, data_size, cols, title, fx, fy, dtype='graph'):
    if data_size > 0:
        rows = (data_size//cols) + 1
        fig, axes = plt.subplots(rows, cols, figsize=(fx,fy))
        fig.suptitle(title,y=1.0,fontsize=40,ha='center')
        ax = axes.ravel()
        for i in range(0, data_size):
            if dtype == 'image':
                ax[i].imshow(data[i])
            else:
                ax[i].plot(data[i])

        for a in ax:
                a.set_axis_off()
                
        plt.tight_layout()
        plt.show()   


In [74]:
def save_files(data,folder,image):
    os.makedirs(folder, exist_ok=True)
    for i in range(0, len(data)):
        img_name = folder + str(i) + ".jpeg"
        image.imsave(img_name, data[i])


In [75]:
def select_model(num_cluster = 1):
    # Extract features of remote sensing imagery of water bodies and 
    # non-water bodies using a pre-trained deep learning architecture.
    resnet = ResNet50V2(include_top=False, pooling='avg', weights='imagenet',input_shape = (64,64,3))#, classes = 3)

    # ResNet model doesn't contain the flattened fully connected layers, so we add them separately.
    resnet.layers

    # ResNet model doesn't contain the flattened fully connected layers, so we add them separately.
    # print(resnet.summary()[-8])
#     print(resnet.layers[-1].output)

    # ResNet model doesn't contain the flattened fully connected layers, so we add them separately.
    ucl_model = Sequential()
    ucl_model.add(resnet)
    ucl_model.add(Flatten())
    ucl_model.add(Dense(128, activation='relu'))
    
    ucl_model.add(Dense(1, activation='softmax'))
      
    # freeze resnet50 conv layers and keep only new layers for fine-tuning. 
    ucl_model.layers[0].trainable = False
#     print(ucl_model.summary())
    return {'ucl_model':ucl_model}   

In [76]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as matplotlib_image
import glob
import os
import cv2
from tensorflow.keras.layers import Flatten, Dense, Input, Lambda, Conv2D, MaxPooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
import tensorflow as tf
import numpy as np
from glob import glob
import os
from tensorflow.keras.models import Sequential
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.utils import shuffle
import re
import cv2
from sklearn.cluster import KMeans
from keras.models import Model
from numpy.linalg import norm

In [77]:
class UnsupervisedCurriculumLearning():
    
    def __init__(self, ucl_model, plt, clusters, selection_threshold, train_epochs, x_train):
        self.ucl_model = ucl_model
        self.clusters = clusters
        self.selection_threshold = selection_threshold
        self.train_epochs = train_epochs   
        self.x_train = x_train

    def step1(self):
        # run pretrained model once at the beginning
        ucl_classes = self.ucl_model.predict(self.x_train)
        # classification results for binary classification of rivers, using imagenet weights
        print("Classes predicted by UCL Model for the first 10 rivers are : ", str(ucl_classes[:10]))
        return {'predicted_classes':ucl_classes}

    def step2(self, plt):
        # Step 2:
        # Create two clusters on the extracted features of remote sensing imagery, assigning 
        # them pseudo-labels of 'static rivers' and 'may meander rivers'

        # Kmeans
        # The images are classified into clusters based on similarity of pixel values. 
        # Each image is assigned a cluster label value given by kmeans.labels_. 
        # So kmeans.labels_ is an array of length 26754 as there are 26754 images in the training set.
        # The input to kmeans comes from the following layer which have imagenet weights after step 1.

        print("Layer type: ", self.ucl_model.layers[-3])
        print("Layer details: ",self.ucl_model.layers[-3].output)

        # create a new model to extract the features from ucl_model. This is the actual input to kmeans.
        # extracted features from the resnet50v2 layer - ucl_model.layers[-3]
        model_features= Model(inputs=self.ucl_model.inputs, outputs=self.ucl_model.layers[-3].output)
        features = model_features.predict(self.x_train)  
#         print(features.shape) # should be (len(x_train), 2048)

        # visualize first 10 images, 3 cols
#         visualize(features, 10, 3, 'Feature Visualization', *(20,20))

        # Initialize the class object
        kmeans = KMeans(n_clusters=self.clusters)

        # Fit kmeans to the labels of clusters.
        kmeans = kmeans.fit(features)

#         print("All Kmeans labels for x_train: ", kmeans.labels_)
#         print("Kmeans cluster centers: ", kmeans.cluster_centers_)

        # filter rows of original data
        tot_clusters = self.clusters
        for i in range(0,tot_clusters) : 
            print("Total number of rivers in cluster", i, ": ",features[kmeans.labels_==i].shape) 
            name = 'Features in Cluster '+str(i)
            # only visualize first 10 rivers in each cluster 
#             visualize(features[kmeans.labels_==i][:10], 10, 3, name, *(20,20))   

        return {'cluster_centers':kmeans.cluster_centers_,'features':features,'kmeans_labels':kmeans.labels_}

    def step3(self, centers, features, kmeans_labels):
        # Step 3 : 
        # From each cluster, select the reliable images using the UCL based selection operation. 
        # Select all features that are at a distance λ from the centroid of the cluster. 
        # The parameter λ is a constant value 
        # that can be adjusted according to the requirement. We have used λ = 0.85 after empirical evaluations.  
        # The closest feature vector to the centroid is considered as a centroid feature vector. 
        # We calculate the similarity between a specific feature vector fi belonging to a cluster k 
        # and the centroid feature vector fk using the inner product. 
        # If the calculated similarity is greater than the λ, the sample of the considered feature vector 
        # is declared as a reliable sample and the cluster label is considered as the pseudo sample label 
        # for the next training cycle.  
        # The number of extracted reliable samples vary at every iteration of fine tuning. 

        # calculate cosine similarity 

        feature_centers = []
        tot_clusters = self.clusters
        
        for cluster in range(0,tot_clusters):
            label_i = features[kmeans_labels == cluster]
            dot_product = np.dot(centers[cluster],label_i.T) #(1,2048).(2048,len(labels0))
            normalize_product = norm(centers[cluster])*norm(label_i,axis=1)
            center_i = dot_product/normalize_product
#             print("Similarity to centroid ",cluster ,":  ",center_i[0:3])
            feature_centers.append(center_i)

        # printing the size of all clusters after selecting using threshold.
        for cluster in range(0, tot_clusters):
            # get selected features
            training_class_i = center_i[center_i >= self.selection_threshold]
#             print("Total features selected for centroid ",cluster,": ",training_class_i.shape)

        # Below is probably the most complicated cell in this entire notebook.
        x_train = []
        y_label = []

        for center in range(0, tot_clusters):        
            # get the indices of the feature maps from first center, selected using the threshold . 
            selected_class_index = np.asarray(np.where(feature_centers[center] >= self.selection_threshold))[0,:]
            # get the indices of the images belonging to the first center
            label_index = np.asarray(np.where(kmeans_labels == center))[0,:]

#             print("Indices belonging to cluster ",center,": ",label_index)
#             print("Indices selected for fine-tuning: ", selected_class_index)
#             print("Indices of the images to be sent to model: ", label_index[selected_class_index])

            # get the selected rivers
            rivers = self.x_train[list(label_index[selected_class_index])] 
            title = "Selected Rivers in cluster " + str(center)
            x_train = x_train + list(rivers)
            y_label = y_label + [center]*len(rivers)
            
            # visualize all rivers in current cluster, 4 cols, 
#             visualize(rivers[:,:,:,1], rivers.shape[0], 4, title, 20, 70, 'image')
#             visualize(rivers[:,:,:,1], 6, 6, title, 20, 70, 'image') # visualize 10 images
           
        # Create the dataset
        x_train = np.array(x_train)
        y_label = np.array(y_label)
        print(f'After selection, there are {len(x_train)} selected from {len(self.x_train)}')
        
        return {'x_train':x_train,'y_label':y_label}

    def step4(self, x_train, y_label):
        # Step 4 : 
        # Fine-tune the deep learning module on reliable samples using pseudo labels given by the clusters. 
        print('in step4, x_train is ', x_train.shape)
        print('in step4, y_label is ', np.array(y_label).shape)
        train = ImageDataGenerator()
        training = train.flow(x_train,y_label,batch_size = 100)
        print("The training data with selected images and predicted label from kmeans is ", training)
        
        self.ucl_model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
       
        history = (self.ucl_model).fit(training, epochs = self.train_epochs, steps_per_epoch=len(training))
        
        # plot the loss history
        
        return self.ucl_model
    
    def run_ucl(self, plt) :
        kmeans_results = self.step2(plt)
        centers = kmeans_results['cluster_centers']
        features = kmeans_results['features']
        kmeans_labels = kmeans_results['kmeans_labels']
        selected_images = self.step3(centers, features, kmeans_labels)
        
        # update ucl_model by updating weights of the new added layers
        self.ucl_model = self.step4(selected_images['x_train'], selected_images['y_label'])
        
        return {'model':self.ucl_model, 'x_train':selected_images['x_train'],'y_label':selected_images['y_label'], 'centers': centers}

In [78]:
folder_path = '../data/UCL_Results_Ying_Task1/'

In [79]:
# helper function to run experiment
def run_exp(experiment, num_clusters, model, threshold, max_iter, Xtrain, max_training_epoches):
    print(f"######### in {experiment} results #########")
    exp = UnsupervisedCurriculumLearning(model, plt, num_clusters, threshold, max_training_epoches, Xtrain)
    # although we focus on the last epoch's result, save all results as well to see if there any difference by training
#     prev_centers = []
#     centers = []
#     clusters = []
#     for i in range(0, max_iter):
#         print(f'starting iteration {i} \n')
#         data = exp.run_ucl(plt)
#         rivers = data['x_train']
#         clusters = data['y_label']
#         prev_centers = centers
#         centers = data['centers']

    # converge when centers converge, or when iterations hit 10.
    past_center = np.array([np.array([0]*2048)]*2)
    curr_center = np.array([np.array([1]*2048)]*2)
    iterations = 0
    while((curr_center == past_center).all() or iterations < max_iter):
        past_center = curr_center
        data = exp.run_ucl(plt)
        curr_center = data['centers']
        iterations = iterations + 1
        print("Iteration is: ",iterations)
        print('\n\n\n')
        
    rivers = data['x_train']
    clusters = data['y_label']

    clusters = np.array(clusters)
    for cluster in range(0,len(np.unique(clusters))):
        results = folder_path + experiment + "/cluster_"+str(cluster)+"/"
        save_files(rivers[np.where(clusters == cluster)[0]], results, matplotlib_image)
        
        
    # save model
    path = results + 'model'
    data['model'].save(path)
    # read model
    mode = model.load_model(path)
        

In [80]:
max_training_epoches = 10 # for model.fit() function
max_iter = 10 # for training, alternative is to check centroid for convergence

In [ ]:
threshold = 0.8
ucl_model2 = select_model(1)['ucl_model']
run_exp('EXP0_thresh80', 2, ucl_model2, threshold, max_iter, X_train, max_training_epoches)

# save the model 
# torch.save(ucl_model2, 'EXP0_thresh80.pth')

# load the model 
# ucl_model2 = torch.load('EXP0_thresh80.pth')

######### in EXP0_thresh80 results #########
Layer type:  <keras.layers.reshaping.flatten.Flatten object at 0x7f12bff6bcd0>
Layer details:  KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='flatten_2/Reshape:0', description="created by layer 'flatten_2'")
772/772 [==============================] - 306s 397ms/step
Total number of rivers in cluster 0 :  (11278, 2048)
Total number of rivers in cluster 1 :  (13404, 2048)
After selection, there are 8777 selected from 24682
in step4, x_train is  (8777, 64, 64, 3)
in step4, y_label is  (8777,)
The training data with selected images and predicted label from kmeans is  <keras.preprocessing.image.NumpyArrayIterator object at 0x7f12cce064d0>
Epoch 1/10
88/88 [==============================] - 109s 1s/step - loss: 0.0905 - accuracy: 0.0967
Epoch 2/10
88/88 [==============================] - 80s 909ms/step - loss: 0.0179 - accuracy: 0.0967
Epoch 3/10
88/88 [==============================] - 149s 2s/step - loss

In [ ]:
## test on the X_test dataset 
pred_classes80 = ucl_model2.predict(X_test)
pred_classes80[pred_classes80 > 0.5] = 1
pred_classes80[pred_classes80 <= 0.5] = 0
# calculate accuracy / confusion matrix etc

In [ ]:
# print(river_labels)
river_labels = [1] * len(pred_classes80)
print(pred_classes80)

In [ ]:
from sklearn.metrics import accuracy_score

# It counts the undefined into consideration, may need to remove those data points for computing accuracy
print(accuracy_score(pred_classes80, river_labels))